Libraries Used

In [2]:
import tensorflow as tf
import numpy as np
import json as js
import os
from keras.models import Model #type: ignore
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Reshape #type: ignore
from keras.optimizers import Adam #type: ignore
from keras.utils import Sequence #type: ignore
from PIL import Image

Loading Paths

In [3]:
with open("config.json",'r') as file:
    paths = js.load(file)

In [ ]:
def CNN_Detector():
    inputs = Input(shape=(256, 256, 3))
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    outputs = MaxPooling2D((2, 2))(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(), loss='mse')
    return model

class ObjectDetectionDataGenerator(Sequence):
    def __init__(self, json_path, image_dir, batch_size=32):
        with open(json_path,'r') as file:
            self.bbox_data = js.load(file)
        self.image_dir = image_dir
        self.batch_size = batch_size
        l = []
        for i in self.bbox_data:
            l.append(i["img_id"])
        self.image_names = l
    
    def __getitem__(self, index):
        batch_images = self.image_names[index * self.batch_size:(index + 1) * self.batch_size]
        images, labels = [], []
        for img_name in batch_images:
            img_path = os.path.join(self.image_dir, img_name)
            

def train_object_detector(model, json_path, image_dir, epochs=10, batch_size=32):
    train_generator = ObjectDetectionDataGenerator(json_path, image_dir, batch_size)
    model.fit(train_generator, epochs=epochs)
    return model

model = CNN_Detector()
trained_model = train_object_detector(model, paths["Preprocessed_Train_json"], paths["Train_resized"])

TypeError: list indices must be integers or slices, not str